# RL Agent - Visualization & Analysis

Advanced visualization of RL agent behavior, learning curves, and performance metrics.

## Features:
- 📈 Learning curves and reward progression
- 🎯 Task completion rates
- 🔄 Action distribution analysis
- 📊 Performance comparison across tasks
- 🎨 Interactive plots with Plotly

## 1. Setup

In [ ]:
!pip install requests pandas matplotlib seaborn plotly numpy

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import sys
import os

# Configuration
BASE_URL = "http://localhost:3001"
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Setup complete!")

## 2. Fetch RL Data

In [ ]:
def fetch_rl_history(limit=100):
    """Fetch episode history from backend"""
    try:
        response = requests.get(f"{BASE_URL}/env/history?limit={limit}")
        if response.status_code == 200:
            return response.json().get('episodes', [])
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

# Fetch data
episodes = fetch_rl_history(limit=100)
df = pd.DataFrame(episodes)

print(f"📊 Loaded {len(df)} episodes")
if len(df) > 0:
    print(f"\nColumns: {df.columns.tolist()}")
    display(df.head())
else:
    print("\n⚠️ No episodes found. Run the RL agent first!")

## 3. Learning Curve - Rewards Over Time

In [ ]:
if len(df) > 0 and 'totalReward' in df.columns:
    # Calculate moving average
    window = 10
    df['reward_ma'] = df['totalReward'].rolling(window=window, min_periods=1).mean()
    
    # Create interactive plot
    fig = go.Figure()
    
    # Raw rewards
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['totalReward'],
        mode='markers',
        name='Raw Reward',
        marker=dict(size=6, opacity=0.5),
        hovertemplate='Episode: %{x}<br>Reward: %{y:.3f}'
    ))
    
    # Moving average
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['reward_ma'],
        mode='lines',
        name=f'{window}-Episode Moving Avg',
        line=dict(width=3, color='red')
    ))
    
    fig.update_layout(
        title='RL Agent Learning Curve',
        xaxis_title='Episode',
        yaxis_title='Total Reward',
        hovermode='x unified',
        height=500
    )
    
    fig.show()
    
    # Statistics
    print("\n📊 Reward Statistics:")
    print(f"Mean: {df['totalReward'].mean():.3f}")
    print(f"Median: {df['totalReward'].median():.3f}")
    print(f"Std Dev: {df['totalReward'].std():.3f}")
    print(f"Min: {df['totalReward'].min():.3f}")
    print(f"Max: {df['totalReward'].max():.3f}")

## 4. Task Completion Analysis

In [ ]:
if len(df) > 0 and 'completed' in df.columns:
    # Success rate over time
    window = 20
    df['success_rate'] = df['completed'].rolling(window=window, min_periods=1).mean() * 100
    
    # Plot
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['success_rate'],
        mode='lines',
        fill='tozeroy',
        name=f'{window}-Episode Success Rate',
        line=dict(width=2, color='green')
    ))
    
    fig.update_layout(
        title='Task Completion Rate Over Time',
        xaxis_title='Episode',
        yaxis_title='Success Rate (%)',
        yaxis_range=[0, 100],
        hovermode='x unified',
        height=400
    )
    
    fig.show()
    
    # Overall stats
    total_completed = df['completed'].sum()
    total_episodes = len(df)
    success_rate = (total_completed / total_episodes * 100)
    
    print(f"\n🎯 Overall Success Rate: {success_rate:.1f}%")
    print(f"Completed: {total_completed}/{total_episodes} episodes")

## 5. Performance by Task Type

In [ ]:
if len(df) > 0 and 'taskType' in df.columns:
    # Group by task type
    task_stats = df.groupby('taskType').agg({
        'completed': ['sum', 'count', 'mean'],
        'totalReward': ['mean', 'std'],
        'steps': 'mean'
    }).round(3)
    
    task_stats.columns = ['Completed', 'Total', 'Success_Rate', 'Avg_Reward', 'Std_Reward', 'Avg_Steps']
    task_stats['Success_Rate'] = task_stats['Success_Rate'] * 100
    
    print("\n📋 Performance by Task Type:")
    display(task_stats)
    
    # Visualize success rate by task
    fig = px.bar(
        task_stats,
        x=task_stats.index,
        y='Success_Rate',
        color='Avg_Reward',
        title='Success Rate and Average Reward by Task Type',
        labels={'Success_Rate': 'Success Rate (%)', 'x': 'Task Type'},
        color_continuous_scale='Viridis',
        text='Success_Rate'
    )
    
    fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    fig.update_layout(height=500)
    fig.show()

## 6. Steps to Completion Analysis

In [ ]:
if len(df) > 0 and 'steps' in df.columns:
    # Box plot by task type
    if 'taskType' in df.columns:
        fig = px.box(
            df,
            x='taskType',
            y='steps',
            color='taskType',
            title='Steps Distribution by Task Type',
            labels={'steps': 'Number of Steps', 'taskType': 'Task Type'},
            points='all'
        )
        
        fig.update_layout(height=500, showlegend=False)
        fig.show()
    
    # Histogram
    plt.figure(figsize=(12, 5))
    plt.hist(df['steps'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
    plt.axvline(df['steps'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["steps"].mean():.1f}')
    plt.axvline(df['steps'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df["steps"].median():.1f}')
    plt.xlabel('Steps to Completion')
    plt.ylabel('Frequency')
    plt.title('Distribution of Steps per Episode')
    plt.legend()
    plt.tight_layout()
    plt.show()

## 7. Reward Distribution

In [ ]:
if len(df) > 0 and 'totalReward' in df.columns and 'taskType' in df.columns:
    # Violin plot
    fig = px.violin(
        df,
        x='taskType',
        y='totalReward',
        color='taskType',
        box=True,
        points='all',
        title='Reward Distribution by Task Type',
        labels={'totalReward': 'Total Reward', 'taskType': 'Task Type'}
    )
    
    fig.update_layout(height=500, showlegend=False)
    fig.show()

## 8. Learning Progress Dashboard

In [ ]:
if len(df) > 0:
    # Create dashboard with subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Rewards Over Time', 'Success Rate', 'Steps Distribution', 'Task Distribution'),
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'histogram'}, {'type': 'pie'}]]
    )
    
    # Subplot 1: Rewards
    if 'totalReward' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['totalReward'], mode='lines', name='Reward'),
            row=1, col=1
        )
    
    # Subplot 2: Success rate
    if 'completed' in df.columns:
        df['success_ma'] = df['completed'].rolling(window=10, min_periods=1).mean() * 100
        fig.add_trace(
            go.Scatter(x=df.index, y=df['success_ma'], mode='lines', name='Success Rate', line=dict(color='green')),
            row=1, col=2
        )
    
    # Subplot 3: Steps histogram
    if 'steps' in df.columns:
        fig.add_trace(
            go.Histogram(x=df['steps'], name='Steps', marker_color='skyblue'),
            row=2, col=1
        )
    
    # Subplot 4: Task distribution
    if 'taskType' in df.columns:
        task_counts = df['taskType'].value_counts()
        fig.add_trace(
            go.Pie(labels=task_counts.index, values=task_counts.values, name='Tasks'),
            row=2, col=2
        )
    
    fig.update_layout(height=800, title_text="RL Agent Performance Dashboard", showlegend=False)
    fig.show()

## 9. Compare First vs Last Episodes

In [ ]:
if len(df) > 20:
    # First 20 episodes
    first_20 = df.head(20)
    last_20 = df.tail(20)
    
    comparison = pd.DataFrame({
        'Metric': ['Avg Reward', 'Success Rate (%)', 'Avg Steps'],
        'First 20': [
            first_20['totalReward'].mean() if 'totalReward' in df.columns else 0,
            first_20['completed'].mean() * 100 if 'completed' in df.columns else 0,
            first_20['steps'].mean() if 'steps' in df.columns else 0
        ],
        'Last 20': [
            last_20['totalReward'].mean() if 'totalReward' in df.columns else 0,
            last_20['completed'].mean() * 100 if 'completed' in df.columns else 0,
            last_20['steps'].mean() if 'steps' in df.columns else 0
        ]
    })
    
    comparison['Improvement'] = comparison['Last 20'] - comparison['First 20']
    comparison['Improvement %'] = (comparison['Improvement'] / comparison['First 20'] * 100).round(1)
    
    print("\n📈 Learning Progress Comparison:")
    print("=" * 70)
    display(comparison)
    
    # Visualize
    fig = go.Figure()
    
    fig.add_trace(go.Bar(name='First 20', x=comparison['Metric'], y=comparison['First 20'], marker_color='lightblue'))
    fig.add_trace(go.Bar(name='Last 20', x=comparison['Metric'], y=comparison['Last 20'], marker_color='darkblue'))
    
    fig.update_layout(
        title='Performance: First 20 vs Last 20 Episodes',
        barmode='group',
        height=400
    )
    
    fig.show()

## 10. Export Visualization Report

In [ ]:
if len(df) > 0:
    # Create summary
    summary = {
        'total_episodes': len(df),
        'avg_reward': df['totalReward'].mean() if 'totalReward' in df.columns else 0,
        'success_rate': df['completed'].mean() * 100 if 'completed' in df.columns else 0,
        'avg_steps': df['steps'].mean() if 'steps' in df.columns else 0,
    }
    
    print("\n" + "=" * 50)
    print("📊 FINAL SUMMARY")
    print("=" * 50)
    for key, value in summary.items():
        print(f"{key}: {value:.2f}" if isinstance(value, float) else f"{key}: {value}")
    print("=" * 50)